<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with SageMaker Machine Learning engine

This notebook works correctly with kernel `Python 3.7.x`. It shows how to log the payload for the model deployed on custom model serving engine using Watson OpenScale python sdk.

Contents
- Setup
- Binding machine learning engine
- Subscriptions
- Performance monitor, scoring and payload logging
- Quality monitor and feedback logging
- Fairness, Drift monitoring and explanations

<a id="setup"></a>
## 1. Setup

### Requirements installation

In [2]:
!pip install sagemaker --no-cache | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1
!pip install --upgrade boto3 --no-cache | tail -n 1
!pip install -U pandas==1.2.5 | tail -n 1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.18.0 requires nose, which is not installed.
hdijupyterutils 0.18.0 requires jupyter>=1, which is not installed.
hdijupyterutils 0.18.0 requires nose, which is not installed.


**Action:** Restart the kernel.

### Sample model creation using [Amazon SageMaker](https://aws.amazon.com/sagemaker/)

- Download this [notebook](https://github.com/IBM/watson-openscale-samples/blob/main/IBM%20Cloud/AWS%20Sagemaker/notebooks/Credit%20%20model%20with%20SageMaker%20linear-learner%20.ipynb) to create SageMaker model
- Run the notebook to train a SageMaker model and create deployment endpoint for online inference

### 1.2 Authentication

Import and initiate.

#### ACTION: Get OpenScale `instance_guid` and `apikey`

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using bluemix console:
```bash
ibmcloud login --sso
ibmcloud iam api-key-create 'my_key'
```

How to get your OpenScale instance GUID

- if your resource group is different than `default` switch to resource group containing OpenScale instance
```bash
ibmcloud target -g <myResourceGroup>
```
- get details of the instance
```bash
ibmcloud resource service-instance 'AI-OpenScale-instance_name'
```

In [26]:
CLOUD_API_KEY = '******'

In [2]:
DB_CREDENTIALS=None
#DB_CREDENTIALS= {"hostname":"","username":"","password":"","database":"","port":"","ssl":True,"sslmode":"","certificate_base64":""}

In [3]:
SCHEMA_NAME = 'data_mart_for_aws_sagemaker'

In [4]:
IAM_URL="https://iam.ng.bluemix.net/oidc/token"
COS_RESOURCE_CRN="<COS_RESOURCE_CRN>"
COS_API_KEY_ID = "<COS_API_KEY_ID>"
COS_ENDPOINT = "https://<URL>" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints

In [16]:
BUCKET_NAME = "******" #example: "<name>-<date>-credit-risk-training-data"
training_data_file_name="credit_risk_training_recoded.csv"

In [7]:
!rm credit_risk_training_recoded.csv
!wget "https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/AWS%20Sagemaker/assets/data/credit_risk_aws/credit_risk_training_recoded.csv"

--2021-08-05 19:03:28--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/AWS%20Sagemaker/assets/data/credit_risk_aws/credit_risk_training_recoded.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 639376 (624K) [text/plain]
Saving to: ‘credit_risk_training_recoded.csv’

credit_risk_trainin 100%[===================>] 624.39K  --.-KB/s    in 0.009s  

2021-08-05 19:03:28 (67.0 MB/s) - ‘credit_risk_training_recoded.csv’ saved [639376/639376]



### Store training data in COS for OpenScale reference

In [21]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

cos_client = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [22]:
BUCKET_NAME

'scottda-aws-openscale-8-5-2021'

In [23]:
with open(training_data_file_name, "rb") as file_data:
    cos_client.Object(BUCKET_NAME, training_data_file_name).upload_fileobj(
        Fileobj=file_data
    )

### Initiate OpenScale client

In [27]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,CloudPakForDataAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
wos_client = APIClient(authenticator=authenticator,service_url="https://aiopenscale.cloud.ibm.com")
wos_client.version

'3.0.9'

Create schema for data mart.

In [28]:
wos_client.data_marts.show()

WOS Data Mart,Data Mart created by WOS tutorial notebook,True,active,2021-07-27 18:41:18.400000+00:00,a08b0e60-6844-4c14-81cb-a3a6c9d199f9


In [29]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL, # For DB2 use DatabaseType.DB2
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS['hostname'],
                        username=DB_CREDENTIALS['username'],
                        password=DB_CREDENTIALS['password'],
                        db=DB_CREDENTIALS['database'],
                        port=DB_CREDENTIALS['port'],
                        ssl=True,
                        sslmode=DB_CREDENTIALS['sslmode'],
                        certificate_base64=DB_CREDENTIALS['certificate_base64']
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

Using existing datamart a08b0e60-6844-4c14-81cb-a3a6c9d199f9


In [30]:
wos_client.data_marts.get(data_mart_id).result.to_dict()

{'metadata': {'id': 'a08b0e60-6844-4c14-81cb-a3a6c9d199f9',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/46816354d9cb773bae86c226aa74c8cc:a08b0e60-6844-4c14-81cb-a3a6c9d199f9:data_mart:a08b0e60-6844-4c14-81cb-a3a6c9d199f9',
  'url': '/v2/data_marts/a08b0e60-6844-4c14-81cb-a3a6c9d199f9',
  'created_at': '2021-07-27T18:41:18.400000Z',
  'created_by': 'IBMid-50M71Q0T71'},
 'entity': {'name': 'WOS Data Mart',
  'description': 'Data Mart created by WOS tutorial notebook',
  'service_instance_crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/46816354d9cb773bae86c226aa74c8cc:a08b0e60-6844-4c14-81cb-a3a6c9d199f9::',
  'internal_database': True,
  'database_configuration': {'database_type': 'postgresql',
   'credentials': {'secret_id': 'b4e5fd2f-5f71-442b-825b-74e3703edea4'},
   'location': {'schema_name': 'a08b0e60-6844-4c14-81cb-a3a6c9d199f9'}},
  'status': {'state': 'active'}}}

<a id="binding"></a>
## 2. Bind machine learning engines

### Bind  `SageMaker` machine learning engine

Provide credentials using following fields:
- `access_key_id`
- `secret_access_key`
- `region`

In [ ]:
SAGEMAKER_ENGINE_CREDENTIALS = {
                   'access_key_id': '<ACCESS_KEY_ID>', 
                   'secret_access_key': '<SECRET_ACCESS_KEY>', 
                   'region': '<REGION>'}

In [32]:
SERVICE_PROVIDER_NAME = "AWS Machine Learning"
SERVICE_PROVIDER_DESCRIPTION = "Added by AWS tutorial WOS notebook."

In [33]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

In [34]:
added_service_provider_result=wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description="AWS Service Provider",
        service_type=ServiceTypes.AMAZON_SAGEMAKER,
        credentials=SageMakerCredentials(
            access_key_id=SAGEMAKER_ENGINE_CREDENTIALS['access_key_id'],
            secret_access_key=SAGEMAKER_ENGINE_CREDENTIALS['secret_access_key'],
            region=SAGEMAKER_ENGINE_CREDENTIALS['region']
        ),
        background_mode=False
    ).result



service_provider_id = added_service_provider_result.metadata.id
print("Service Provider id ", service_provider_id)




 Waiting for end of adding service provider 1f0fa78b-7f25-402a-85df-6ff7016848c7 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------


Service Provider id  1f0fa78b-7f25-402a-85df-6ff7016848c7


In [36]:
wos_client.service_providers.show()

None,active,AWS Machine Learning,amazon_sagemaker,2021-08-05 19:15:49.219000+00:00,1f0fa78b-7f25-402a-85df-6ff7016848c7
None,active,Azure Machine Learning,azure_machine_learning,2021-07-29 12:50:37.092000+00:00,ba1b62a0-2a80-414e-9177-bcf71d5a8909


In [35]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id).result
asset_deployment_details

{'resources': [{'metadata': {'guid': '44cbb38bb2c81f10236feaff987a5bb9',
    'url': 'Credit-risk-endpoint-scoring-2021-08-05-17-39',
    'created_at': '2021-08-05T18:04:15.144Z',
    'modified_at': '2021-08-05T18:11:23.172Z'},
   'entity': {'name': 'Credit-risk-endpoint-scoring-2021-08-05-17-39',
    'deployment_rn': 'arn:aws:sagemaker:us-east-2:593389159126:endpoint/credit-risk-endpoint-scoring-2021-08-05-17-39',
    'type': 'online',
    'scoring_endpoint': {'url': 'Credit-risk-endpoint-scoring-2021-08-05-17-39'},
    'asset': {'asset_id': '9345b2a3d122475ee4ea44be26d16df4',
     'asset_rn': 'arn:aws:sagemaker:us-east-2:593389159126:model/credit-risk-linear-learner-2021-08-05-17-39',
     'url': 's3://sagemaker-us-east-2-593389159126/credit-risk/output/Credit-risk-linear-learner-2021-08-05-17-39/output/model.tar.gz',
     'name': 'Credit-risk-linear-learner-2021-08-05-17-39',
     'asset_type': 'model',
     'created_at': '2021-08-05T18:04:07.177Z'},
    'asset_properties': {'asset_r

In [37]:
deployment_id='44cbb38bb2c81f10236feaff987a5bb9' # Get GUID for the deployment created using the other notebook "Credit model with SageMaker linear-learner.ipynb"
for model_asset_details in asset_deployment_details['resources']:
    if model_asset_details['metadata']['guid']==deployment_id:
        break
model_asset_details

{'metadata': {'guid': '44cbb38bb2c81f10236feaff987a5bb9',
  'url': 'Credit-risk-endpoint-scoring-2021-08-05-17-39',
  'created_at': '2021-08-05T18:04:15.144Z',
  'modified_at': '2021-08-05T18:11:23.172Z'},
 'entity': {'name': 'Credit-risk-endpoint-scoring-2021-08-05-17-39',
  'deployment_rn': 'arn:aws:sagemaker:us-east-2:593389159126:endpoint/credit-risk-endpoint-scoring-2021-08-05-17-39',
  'type': 'online',
  'scoring_endpoint': {'url': 'Credit-risk-endpoint-scoring-2021-08-05-17-39'},
  'asset': {'asset_id': '9345b2a3d122475ee4ea44be26d16df4',
   'asset_rn': 'arn:aws:sagemaker:us-east-2:593389159126:model/credit-risk-linear-learner-2021-08-05-17-39',
   'url': 's3://sagemaker-us-east-2-593389159126/credit-risk/output/Credit-risk-linear-learner-2021-08-05-17-39/output/model.tar.gz',
   'name': 'Credit-risk-linear-learner-2021-08-05-17-39',
   'asset_type': 'model',
   'created_at': '2021-08-05T18:04:07.177Z'},
  'asset_properties': {'asset_revision': '1628187083172'}}}

<a id="subsciption"></a>
## 3. Subscriptions

### Add subscriptions

List available deployments.

**Note:** Depending on number of assets it may take some time.

In [38]:
aws_asset = Asset(
        asset_id=model_asset_details['entity']['asset']['asset_id'],
        name=model_asset_details['entity']['asset']['name'],
        url=model_asset_details['entity']['asset']['url'],
        asset_type=model_asset_details['entity']['asset']['asset_type'] if 'asset_type' in model_asset_details['entity']['asset'] else 'model',
        problem_type=ProblemType.BINARY_CLASSIFICATION,
        input_data_type=InputDataType.STRUCTURED,
    )

In [39]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import ScoringEndpointRequest
deployment_scoring_endpoint = model_asset_details['entity']['scoring_endpoint']
scoring_endpoint = ScoringEndpointRequest(url = model_asset_details['entity']['scoring_endpoint']['url'] )

In [40]:
deployment = AssetDeploymentRequest(
        deployment_id=model_asset_details['metadata']['guid'],
        url=model_asset_details['metadata']['url'],
        name=model_asset_details['entity']['name'],
        deployment_type=model_asset_details['entity']['type'],
        scoring_endpoint =  scoring_endpoint
    )

In [41]:
training_data_reference = TrainingDataReference(type='cos',
                                              location=COSTrainingDataReferenceLocation(bucket = BUCKET_NAME,
                                                                                        file_name = training_data_file_name),
                                              connection=COSTrainingDataReferenceConnection(
                                                                        resource_instance_id= COS_RESOURCE_CRN,
                                                                        url= COS_ENDPOINT,
                                                                        api_key= COS_API_KEY_ID,
                                                                        iam_url=IAM_URL)
                                               )

In [42]:
feature_columns = ['CheckingStatus_0_to_200', 'CheckingStatus_greater_200', 'CheckingStatus_less_0', 'CheckingStatus_no_checking', 'CreditHistory_all_credits_paid_back', 'CreditHistory_credits_paid_to_date', 'CreditHistory_no_credits', 'CreditHistory_outstanding_credit', 'CreditHistory_prior_payments_delayed', 'LoanPurpose_appliances', 'LoanPurpose_business', 'LoanPurpose_car_new', 'LoanPurpose_car_used', 'LoanPurpose_education', 'LoanPurpose_furniture', 'LoanPurpose_other', 'LoanPurpose_radio_tv', 'LoanPurpose_repairs', 'LoanPurpose_retraining', 'LoanPurpose_vacation', 'ExistingSavings_100_to_500', 'ExistingSavings_500_to_1000', 'ExistingSavings_greater_1000', 'ExistingSavings_less_100', 'ExistingSavings_unknown', 'EmploymentDuration_1_to_4', 'EmploymentDuration_4_to_7', 'EmploymentDuration_greater_7', 'EmploymentDuration_less_1', 'EmploymentDuration_unemployed', 'Sex_female', 'Sex_male', 'OthersOnLoan_co-applicant', 'OthersOnLoan_guarantor', 'OthersOnLoan_none', 'OwnsProperty_car_other', 'OwnsProperty_real_estate', 'OwnsProperty_savings_insurance', 'OwnsProperty_unknown', 'InstallmentPlans_bank', 'InstallmentPlans_none', 'InstallmentPlans_stores', 'Housing_free', 'Housing_own', 'Housing_rent', 'Job_management_self-employed', 'Job_skilled', 'Job_unemployed', 'Job_unskilled', 'Telephone_none', 'Telephone_yes', 'ForeignWorker_no', 'ForeignWorker_yes', 'LoanDuration', 'LoanAmount', 'InstallmentPercent', 'CurrentResidenceDuration', 'Age', 'ExistingCreditsCount', 'Dependents']
categorical_columns = ['CheckingStatus_0_to_200', 'CheckingStatus_greater_200', 'CheckingStatus_less_0', 'CheckingStatus_no_checking', 'CreditHistory_all_credits_paid_back', 'CreditHistory_credits_paid_to_date', 'CreditHistory_no_credits', 'CreditHistory_outstanding_credit', 'CreditHistory_prior_payments_delayed', 'LoanPurpose_appliances', 'LoanPurpose_business', 'LoanPurpose_car_new', 'LoanPurpose_car_used', 'LoanPurpose_education', 'LoanPurpose_furniture', 'LoanPurpose_other', 'LoanPurpose_radio_tv', 'LoanPurpose_repairs', 'LoanPurpose_retraining', 'LoanPurpose_vacation', 'ExistingSavings_100_to_500', 'ExistingSavings_500_to_1000', 'ExistingSavings_greater_1000', 'ExistingSavings_less_100', 'ExistingSavings_unknown', 'EmploymentDuration_1_to_4', 'EmploymentDuration_4_to_7', 'EmploymentDuration_greater_7', 'EmploymentDuration_less_1', 'EmploymentDuration_unemployed', 'Sex_female', 'Sex_male', 'OthersOnLoan_co-applicant', 'OthersOnLoan_guarantor', 'OthersOnLoan_none', 'OwnsProperty_car_other', 'OwnsProperty_real_estate', 'OwnsProperty_savings_insurance', 'OwnsProperty_unknown', 'InstallmentPlans_bank', 'InstallmentPlans_none', 'InstallmentPlans_stores', 'Housing_free', 'Housing_own', 'Housing_rent', 'Job_management_self-employed', 'Job_skilled', 'Job_unemployed', 'Job_unskilled', 'Telephone_none', 'Telephone_yes', 'ForeignWorker_no', 'ForeignWorker_yes']

In [43]:
asset_properties = AssetPropertiesRequest(
        label_column="Risk",
        prediction_field='predicted_label',
        probability_fields=['score'],
        training_data_reference=training_data_reference,
        training_data_schema=None,
        input_data_schema=None,
        output_data_schema=None,
        feature_fields=feature_columns,
        categorical_fields=categorical_columns
    )

In [44]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=aws_asset,
        deployment=deployment,
        asset_properties=asset_properties,
        background_mode=False
).result
subscription_id = subscription_details.metadata.id
subscription_id




 Waiting for end of adding subscription c45582ad-79dc-42ef-8c43-496042f1e532 




active

-------------------------------------------
 Successfully finished adding subscription 
-------------------------------------------




'c45582ad-79dc-42ef-8c43-496042f1e532'

#### List subscriptions

In [45]:
wos_client.subscriptions.show()

9345b2a3d122475ee4ea44be26d16df4,Credit-risk-linear-learner-2021-08-05-17-39,a08b0e60-6844-4c14-81cb-a3a6c9d199f9,44cbb38bb2c81f10236feaff987a5bb9,Credit-risk-endpoint-scoring-2021-08-05-17-39,1f0fa78b-7f25-402a-85df-6ff7016848c7,active,2021-08-05 19:18:49.703000+00:00,c45582ad-79dc-42ef-8c43-496042f1e532
6de5e798b8b64cc5d09e91734a149197,DaveGermanCredit.2019.2.12.17.16.52.617,a08b0e60-6844-4c14-81cb-a3a6c9d199f9,9764f091ac3099d6c5182764c7b97f90,DaveGermanCredit.2019.2.12.17.16.52.617,ba1b62a0-2a80-414e-9177-bcf71d5a8909,active,2021-07-29 12:51:37.381000+00:00,62d30e9f-8fdc-4006-88f5-f76a7edb72e7


<a id="scoring"></a>
## 4. Performance metrics, scoring and payload logging

### Score the credit risk model and measure response time

In [46]:
import requests
import time
import json
import boto3

In [47]:
subscription_details=wos_client.subscriptions.get(subscription_id).result.to_dict()
subscription_details

{'metadata': {'id': 'c45582ad-79dc-42ef-8c43-496042f1e532',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/46816354d9cb773bae86c226aa74c8cc:a08b0e60-6844-4c14-81cb-a3a6c9d199f9:subscription:c45582ad-79dc-42ef-8c43-496042f1e532',
  'url': '/v2/subscriptions/c45582ad-79dc-42ef-8c43-496042f1e532',
  'created_at': '2021-08-05T19:18:49.703000Z',
  'created_by': 'IBMid-50M71Q0T71'},
 'entity': {'data_mart_id': 'a08b0e60-6844-4c14-81cb-a3a6c9d199f9',
  'service_provider_id': '1f0fa78b-7f25-402a-85df-6ff7016848c7',
  'asset': {'asset_id': '9345b2a3d122475ee4ea44be26d16df4',
   'url': 's3://sagemaker-us-east-2-593389159126/credit-risk/output/Credit-risk-linear-learner-2021-08-05-17-39/output/model.tar.gz',
   'name': 'Credit-risk-linear-learner-2021-08-05-17-39',
   'asset_type': 'model',
   'problem_type': 'binary',
   'input_data_type': 'structured'},
  'asset_properties': {'training_data_reference': {'secret_id': '3e458517-66bd-422c-8056-77fc89619d52'},
   'output_data_schema': {'typ

In [49]:
endpoint_name = subscription_details['entity']['deployment']['name']

payload = "0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,12,4152,2,3,29,2,1"

In [50]:
runtime = boto3.client('sagemaker-runtime',
                       region_name=SAGEMAKER_ENGINE_CREDENTIALS['region'],
                       aws_access_key_id=SAGEMAKER_ENGINE_CREDENTIALS['access_key_id'],
                       aws_secret_access_key=SAGEMAKER_ENGINE_CREDENTIALS['secret_access_key'])

start_time = time.time()
response = runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=payload)
response_time = int((time.time() - start_time)*1000)
result = json.loads(response['Body'].read().decode())

print(json.dumps(result, indent=2))

{
  "predictions": [
    {
      "score": 0.19936884939670563,
      "predicted_label": 0
    }
  ]
}


### Store the request and response in payload logging table

#### Transform the model's input and output to the format compatible with OpenScale standard.

In [51]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

Payload data set id:  2442cacc-5dcf-4117-adc8-d9e3c6b0e38c


In [52]:
values = [float(s) for s in payload.split(',')]

request_data = {'fields': feature_columns, 
                'values': values}

response_data = {'fields': list(result['predictions'][0]),
                 'values': [list(x.values()) for x in result['predictions']]}

#### Store the payload using Python SDK

**Hint:** You can embed payload logging code into your custom deployment so it is logged automatically each time you score the model.

In [53]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

print("Performing explicit payload logging.....")
wos_client.data_sets.store_records(data_set_id=payload_data_set_id, background_mode=False,request_body=[PayloadRecord(
           scoring_id=str(uuid.uuid4()),
           request=request_data,
           response=response_data,
           response_time=460
)])
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

Performing explicit payload logging.....



 Waiting for end of storing records with request id: e3a80ce7-a2c9-4e46-9f28-2d9b2ab23120 




active

---------------------------------------
 Successfully finished storing records 
---------------------------------------


Number of records in the payload logging table: 1


In [54]:
wos_client.data_sets.show_records(data_set_id=payload_data_set_id)

1.0,None,0.0,0.0,0.0,0.0,4152.0,6c4fc7fa-50b8-4ecb-b82d-9163f48c0c1c-1,0.0,0.0,1.0,2.0,0,0.0,2021-08-05T19:20:35.446Z,1.0,1.0,0.0,0.8006311506032944,0.0,"[0.8006311506032944, 0.19936884939670563]",0.0,0.0,0.19936884939670563,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,12.0,29.0,0.0,1.0,3.0,0.0,0.0,0.0,1.0,0.0,2.0,44cbb38bb2c81f10236feaff987a5bb9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


<a id="feedback"></a>
## 5. Feedback logging & quality (accuracy) monitoring

### Enable quality monitoring

You need to provide the monitoring `threshold` and `min_records` (minimal number of feedback records).

In [55]:
import time

time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 10
}
thresholds = [
                {
                    "metric_id": "area_under_roc",
                    "type": "lower_limit",
                    "value": .80
                }
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result




 Waiting for end of monitor instance creation ea7f0dbc-7d1c-4b0f-9ab6-553bbb6478ed 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [56]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

'ea7f0dbc-7d1c-4b0f-9ab6-553bbb6478ed'

### Feedback records logging

Feedback records are used to evaluate your model. The predicted values are compared to real values (feedback records).

You can check the schema of feedback table using below method.

In [57]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")
feedback_dataset_id

'4bcfedec-459d-4658-9673-40a6dcbe0883'

The feedback records can be send to feedback table using below code.

In [58]:
import requests
import pandas as pd
import numpy as np
import time

time.sleep(10) #It gives enough time for dataset creation

data = pd.read_csv('https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/AWS%20Sagemaker/assets/data/credit_risk_aws/credit_risk_feedback_recoded.csv',header=0,dtype=np.float)
feedback_columns = data.columns.tolist()
feedback_records = data.values.tolist()

payload_scoring =  [{"fields": feedback_columns, "values": feedback_records}]
wos_client.data_sets.store_records(feedback_dataset_id, request_body=payload_scoring, background_mode=False)




 Waiting for end of storing records with request id: 15a3d5a4-c4fb-4604-8829-b68481b0e6b3 




active

---------------------------------------
 Successfully finished storing records 
---------------------------------------




In [59]:
wos_client.data_sets.print_records_schema(data_set_id=feedback_dataset_id)

CheckingStatus_0_to_200,double,True
CheckingStatus_greater_200,double,True
CheckingStatus_less_0,double,True
CheckingStatus_no_checking,double,True
CreditHistory_all_credits_paid_back,double,True
CreditHistory_credits_paid_to_date,double,True
CreditHistory_no_credits,double,True
CreditHistory_outstanding_credit,double,True
CreditHistory_prior_payments_delayed,double,True
LoanPurpose_appliances,double,True
LoanPurpose_business,double,True


In [60]:
wos_client.data_sets.get_records_count(data_set_id=feedback_dataset_id)

20

In [61]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result




 Waiting for end of monitoring run 80bebde4-7577-4db3-b525-0738446cbbef 




running
finished

---------------------------
 Successfully finished run 
---------------------------




In [62]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

2021-08-05 19:23:20.156000+00:00,true_positive_rate,c65c89c2-76b4-433e-8cc0-5802b6d1b490,0.7142857142857143,None,None,['model_type:original'],quality,ea7f0dbc-7d1c-4b0f-9ab6-553bbb6478ed,80bebde4-7577-4db3-b525-0738446cbbef,subscription,c45582ad-79dc-42ef-8c43-496042f1e532
2021-08-05 19:23:20.156000+00:00,area_under_roc,c65c89c2-76b4-433e-8cc0-5802b6d1b490,0.8186813186813188,0.8,None,['model_type:original'],quality,ea7f0dbc-7d1c-4b0f-9ab6-553bbb6478ed,80bebde4-7577-4db3-b525-0738446cbbef,subscription,c45582ad-79dc-42ef-8c43-496042f1e532
2021-08-05 19:23:20.156000+00:00,precision,c65c89c2-76b4-433e-8cc0-5802b6d1b490,0.8333333333333334,None,None,['model_type:original'],quality,ea7f0dbc-7d1c-4b0f-9ab6-553bbb6478ed,80bebde4-7577-4db3-b525-0738446cbbef,subscription,c45582ad-79dc-42ef-8c43-496042f1e532
2021-08-05 19:23:20.156000+00:00,f1_measure,c65c89c2-76b4-433e-8cc0-5802b6d1b490,0.7692307692307692,None,None,['model_type:original'],quality,ea7f0dbc-7d1c-4b0f-9ab6-553bbb6478ed,80bebde4-7577-4db3-b525-0738446cbbef,subscription,c45582ad-79dc-42ef-8c43-496042f1e532
2021-08-05 19:23:20.156000+00:00,accuracy,c65c89c2-76b4-433e-8cc0-5802b6d1b490,0.85,None,None,['model_type:original'],quality,ea7f0dbc-7d1c-4b0f-9ab6-553bbb6478ed,80bebde4-7577-4db3-b525-0738446cbbef,subscription,c45582ad-79dc-42ef-8c43-496042f1e532
2021-08-05 19:23:20.156000+00:00,log_loss,c65c89c2-76b4-433e-8cc0-5802b6d1b490,0.3957367345523932,None,None,['model_type:original'],quality,ea7f0dbc-7d1c-4b0f-9ab6-553bbb6478ed,80bebde4-7577-4db3-b525-0738446cbbef,subscription,c45582ad-79dc-42ef-8c43-496042f1e532
2021-08-05 19:23:20.156000+00:00,false_positive_rate,c65c89c2-76b4-433e-8cc0-5802b6d1b490,0.07692307692307693,None,None,['model_type:original'],quality,ea7f0dbc-7d1c-4b0f-9ab6-553bbb6478ed,80bebde4-7577-4db3-b525-0738446cbbef,subscription,c45582ad-79dc-42ef-8c43-496042f1e532
2021-08-05 19:23:20.156000+00:00,area_under_pr,c65c89c2-76b4-433e-8cc0-5802b6d1b490,0.7642857142857142,None,None,['model_type:original'],quality,ea7f0dbc-7d1c-4b0f-9ab6-553bbb6478ed,80bebde4-7577-4db3-b525-0738446cbbef,subscription,c45582ad-79dc-42ef-8c43-496042f1e532
2021-08-05 19:23:20.156000+00:00,recall,c65c89c2-76b4-433e-8cc0-5802b6d1b490,0.7142857142857143,None,None,['model_type:original'],quality,ea7f0dbc-7d1c-4b0f-9ab6-553bbb6478ed,80bebde4-7577-4db3-b525-0738446cbbef,subscription,c45582ad-79dc-42ef-8c43-496042f1e532


<a id="datamart"></a>
## 6. Get the logged data

### Payload logging

#### Print schema of payload_logging table

In [63]:
wos_client.data_sets.print_records_schema(data_set_id=payload_data_set_id)

scoring_id,string,False
scoring_timestamp,timestamp,False
deployment_id,string,False
asset_revision,string,True
CheckingStatus_0_to_200,double,True
CheckingStatus_greater_200,double,True
CheckingStatus_less_0,double,True
CheckingStatus_no_checking,double,True
CreditHistory_all_credits_paid_back,double,True
CreditHistory_credits_paid_to_date,double,True
CreditHistory_no_credits,double,True


<a id="fairness_and_explainability"></a>
## 7. Fairness, Drift monitoring and explanations

### Get payload data

In [64]:
scoring_data_filename='credit_risk_scoring_recoded.csv'
scoring_data_filename_json='credit_risk_scoring_recoded.json'

In [65]:
!rm credit_risk_scoring_recoded.json
!rm credit_risk_scoring_recoded.csv
!wget "https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/AWS%20Sagemaker/assets/data/credit_risk_aws/credit_risk_scoring_recoded.json"
!wget "https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/AWS%20Sagemaker/assets/data/credit_risk_aws/credit_risk_scoring_recoded.csv"

rm: cannot remove 'credit_risk_scoring_recoded.json': No such file or directory
--2021-08-05 19:23:51--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/AWS%20Sagemaker/assets/data/credit_risk_aws/credit_risk_scoring_recoded.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5108 (5.0K) [text/plain]
Saving to: ‘credit_risk_scoring_recoded.json’

credit_risk_scoring 100%[===================>]   4.99K  --.-KB/s    in 0s      

2021-08-05 19:23:51 (34.1 MB/s) - ‘credit_risk_scoring_recoded.json’ saved [5108/5108]

--2021-08-05 19:23:52--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/AWS%20Sagemaker/assets/data/credit_risk_aws/credit_risk_scoring_recoded.csv
Resolving raw.githubusercontent.com 

In [66]:
sm_runtime = boto3.client('sagemaker-runtime',
                       region_name=SAGEMAKER_ENGINE_CREDENTIALS['region'],
                       aws_access_key_id=SAGEMAKER_ENGINE_CREDENTIALS['access_key_id'],
                       aws_secret_access_key=SAGEMAKER_ENGINE_CREDENTIALS['secret_access_key'])

with open(scoring_data_filename) as f_payload:
    scoring_response = sm_runtime.invoke_endpoint(EndpointName = endpoint_name,
                                                  ContentType = 'text/csv',
                                                  Body = f_payload.read().encode())
    
    result = json.loads(scoring_response['Body'].read().decode())
    print(json.dumps(result, indent=2))

{
  "predictions": [
    {
      "score": 0.44011253118515015,
      "predicted_label": 0
    },
    {
      "score": 0.9322102069854736,
      "predicted_label": 1
    },
    {
      "score": 0.23179161548614502,
      "predicted_label": 0
    },
    {
      "score": 0.5139932632446289,
      "predicted_label": 0
    },
    {
      "score": 0.11651258915662766,
      "predicted_label": 0
    },
    {
      "score": 0.2591845691204071,
      "predicted_label": 0
    },
    {
      "score": 0.09976425766944885,
      "predicted_label": 0
    },
    {
      "score": 0.22220531105995178,
      "predicted_label": 0
    },
    {
      "score": 0.02672547660768032,
      "predicted_label": 0
    },
    {
      "score": 0.048588234931230545,
      "predicted_label": 0
    },
    {
      "score": 0.14364618062973022,
      "predicted_label": 0
    },
    {
      "score": 0.4464980959892273,
      "predicted_label": 0
    },
    {
      "score": 0.6185409426689148,
      "predicted_label": 1
  

In [67]:
f = open(scoring_data_filename_json,"r")
payload_values = json.load(f)
request_data = {'fields': feature_columns, 
                'values': payload_values}

response_data = {'fields': list(result['predictions'][0]),
                 'values': [list(x.values()) for x in result['predictions']]}


In [68]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

print("Performing explicit payload logging.....")
wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
           scoring_id=str(uuid.uuid4()),
           request=request_data,
           response=response_data,
           response_time=460
)])
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

Performing explicit payload logging.....
Number of records in the payload logging table: 41


In [69]:
wos_client.data_sets.show_records(payload_data_set_id)

1.0,None,0.0,0.0,0.0,0.0,5308.0,97093668-aebf-4167-8988-d85ac19127a9-1,0.0,0.0,1.0,3.0,0,0.0,2021-08-05T19:24:01.277Z,0.0,1.0,0.0,0.5598874688148499,0.0,"[0.5598874688148499, 0.44011253118515015]",0.0,0.0,0.44011253118515015,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,38.0,31.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,44cbb38bb2c81f10236feaff987a5bb9,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
0.0,None,0.0,0.0,0.0,0.0,620.0,97093668-aebf-4167-8988-d85ac19127a9-10,0.0,0.0,0.0,2.0,0,0.0,2021-08-05T19:24:01.277Z,0.0,1.0,0.0,0.9514117650687695,0.0,"[0.9514117650687695, 0.048588234931230545]",0.0,0.0,0.048588234931230545,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,12.0,30.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,44cbb38bb2c81f10236feaff987a5bb9,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
0.0,None,0.0,0.0,0.0,0.0,1387.0,97093668-aebf-4167-8988-d85ac19127a9-11,0.0,0.0,1.0,2.0,0,0.0,2021-08-05T19:24:01.277Z,1.0,1.0,0.0,0.8563538193702698,0.0,"[0.8563538193702698, 0.14364618062973022]",0.0,0.0,0.14364618062973022,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,16.0,28.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,44cbb38bb2c81f10236feaff987a5bb9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1.0,None,0.0,0.0,0.0,0.0,6225.0,97093668-aebf-4167-8988-d85ac19127a9-12,0.0,0.0,1.0,3.0,0,0.0,2021-08-05T19:24:01.277Z,0.0,1.0,0.0,0.5535019040107727,0.0,"[0.5535019040107727, 0.4464980959892273]",0.0,0.0,0.4464980959892273,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,25.0,40.0,1.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,44cbb38bb2c81f10236feaff987a5bb9,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1.0,None,0.0,1.0,0.0,0.0,4424.0,97093668-aebf-4167-8988-d85ac19127a9-13,0.0,0.0,1.0,3.0,1,0.0,2021-08-05T19:24:01.277Z,0.0,1.0,0.0,0.6185409426689148,0.0,"[0.3814590573310852, 0.6185409426689148]",0.0,0.0,0.6185409426689148,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,32.0,32.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,2.0,44cbb38bb2c81f10236feaff987a5bb9,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
0.0,None,0.0,0.0,1.0,0.0,2244.0,97093668-aebf-4167-8988-d85ac19127a9-14,0.0,0.0,1.0,2.0,0,0.0,2021-08-05T19:24:01.277Z,1.0,1.0,0.0,0.9773842208087444,0.0,"[0.9773842208087444, 0.02261577919125557]",0.0,0.0,0.02261577919125557,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,14.0,21.0,0.0,1.0,3.0,0.0,0.0,0.0,1.0,0.0,1.0,44cbb38bb2c81f10236feaff987a5bb9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1.0,None,0.0,1.0,0.0,0.0,2992.0,97093668-aebf-4167-8988-d85ac19127a9-15,0.0,0.0,1.0,4.0,1,0.0,2021-08-05T19:24:01.277Z,0.0,1.0,0.0,0.6230925917625427,0.0,"[0.3769074082374573, 0.6230925917625427]",0.0,0.0,0.6230925917625427,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,53.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,2.0,44cbb38bb2c81f10236feaff987a5bb9,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
1.0,None,0.0,1.0,0.0,0.0,2034.0,97093668-aebf-4167-8988-d85ac19127a9-16,0.0,0.0,1.0,4.0,0,0.0,2021-08-05T19:24:01.277Z,0.0,1.0,0.0,0.6244255900382996,0.0,"[0.6244255900382996, 0.37557440996170044]",0.0,0.0,0.37557440996170044,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,47.0,0.0,1.0,3.0,0.0,0.0,0.0,1.0,1.0,2.0,44cbb38bb2c81f10236feaff987a5bb9,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
0.0,None,0.0,0.0,1.0,0.0,2804.0,97093668-aebf-4167-8988-d85ac19127a9-17,0.0,0.0,1.0,3.0,0,0.0,2021-08-05T19:24:01.277Z,0.0,1.0,0.0,0.9635847955942154,0.0,"[0.9635847955942154, 0.03641520440578461]",0.0,0.0,0.03641520440578461,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,19.0,31.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,44cbb38bb2c81f10236feaff987a5bb9,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
0.0,None,0.0,0.0,1.0,0.0,259.0,97093668-aebf-4167-8988-d85ac19127a9-18,0.0,0.0,1.0,2.0,0,0.0,2021-08-05T19:24

### Enable and run fairness monitoring

In [70]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "features": [
        {"feature": "Sex_female",
         "majority": [[0,0]],
         "minority": [[1,1]],
         "threshold": 0.95
         },
        {"feature": "Age",
         "majority": [[26, 75]],
         "minority": [[18, 25]],
         "threshold": 0.95
         }
    ],
    "favourable_class": [0],
    "unfavourable_class": [1],
    "min_records": 30
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id
fairness_monitor_instance_id




 Waiting for end of monitor instance creation 8912f12b-95e5-4ea8-999d-1f5eabe1c191 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'8912f12b-95e5-4ea8-999d-1f5eabe1c191'

### Run fairness monitor

In [71]:
time.sleep(20)
#Note: When you create fairness monitor, initial run is also created
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

2021-08-05 19:24:19.411806+00:00,fairness_value,10f4442e-96bc-4329-b293-f3a903411462,120.0,80.0,None,"['feature:Sex_female', 'fairness_metric_type:fairness', 'feature_value:1-1']",fairness,8912f12b-95e5-4ea8-999d-1f5eabe1c191,1b1c5bba-460d-498e-88c1-218af65132d9,subscription,c45582ad-79dc-42ef-8c43-496042f1e532
2021-08-05 19:24:19.411806+00:00,fairness_value,10f4442e-96bc-4329-b293-f3a903411462,108.0,80.0,None,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,8912f12b-95e5-4ea8-999d-1f5eabe1c191,1b1c5bba-460d-498e-88c1-218af65132d9,subscription,c45582ad-79dc-42ef-8c43-496042f1e532
2021-08-05 19:24:19.411806+00:00,fairness_value,ae1d38e4-2e5e-4190-8325-5b5bf66dd5a3,120.0,80.0,None,"['feature:Sex_female', 'fairness_metric_type:debiased_fairness', 'feature_value:1-1']",fairness,8912f12b-95e5-4ea8-999d-1f5eabe1c191,1b1c5bba-460d-498e-88c1-218af65132d9,subscription,c45582ad-79dc-42ef-8c43-496042f1e532
2021-08-05 19:24:19.411806+00:00,fairness_value,ae1d38e4-2e5e-4190-8325-5b5bf66dd5a3,108.0,80.0,None,"['feature:Age', 'fairness_metric_type:debiased_fairness', 'feature_value:18-25']",fairness,8912f12b-95e5-4ea8-999d-1f5eabe1c191,1b1c5bba-460d-498e-88c1-218af65132d9,subscription,c45582ad-79dc-42ef-8c43-496042f1e532


### Enable and run Drift monitoring

#### Drift requires a trained model to be uploaded manually for AWS. You can train, create and download a drift detection model using template given ( check for Drift detection model generation) [here](https://github.com/IBM-Watson/aios-data-distribution/blob/master/training_statistics_notebook.ipynb)

In [72]:
!rm -rf creditrisk_aws_drift_detection_model.tar.gz
!wget -O creditrisk_aws_drift_detection_model.tar.gz https://github.com/IBM/watson-openscale-samples/blob/main/IBM%20Cloud/AWS%20Sagemaker/assets/models/credit_risk/aws_creditrisk_drift_detection_model.tar.gz?raw=true 

--2021-08-05 19:25:14--  https://github.com/IBM/watson-openscale-samples/blob/main/IBM%20Cloud/AWS%20Sagemaker/assets/models/credit_risk/aws_creditrisk_drift_detection_model.tar.gz?raw=true
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/IBM/watson-openscale-samples/raw/main/IBM%20Cloud/AWS%20Sagemaker/assets/models/credit_risk/aws_creditrisk_drift_detection_model.tar.gz [following]
--2021-08-05 19:25:14--  https://github.com/IBM/watson-openscale-samples/raw/main/IBM%20Cloud/AWS%20Sagemaker/assets/models/credit_risk/aws_creditrisk_drift_detection_model.tar.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/AWS%20Sagemaker/assets/models/credit_risk/aws_creditrisk_drift_detection_model.tar.gz [following

In [73]:
wos_client.monitor_instances.upload_drift_model(
        model_path='creditrisk_aws_drift_detection_model.tar.gz',
        data_mart_id=data_mart_id,
        subscription_id=subscription_id
     )

In [74]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)

In [75]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "min_samples": 30,
    "drift_threshold": 0.1,
    "train_drift_model": False,
    "enable_model_drift": True,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id




 Waiting for end of monitor instance creation ce617984-9b10-4a73-9fa6-8cfcaa027e49 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'ce617984-9b10-4a73-9fa6-8cfcaa027e49'

In [76]:
drift_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run 1e9e9530-4679-48d3-a06a-a903b9279b4c 




running
finished

---------------------------
 Successfully finished run 
---------------------------




In [77]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

2021-08-05 19:25:41.218924+00:00,data_drift_magnitude,45ab47c1-6d1d-4ac1-8f00-f27c73bbdb46,1.0,None,None,[],drift,ce617984-9b10-4a73-9fa6-8cfcaa027e49,1e9e9530-4679-48d3-a06a-a903b9279b4c,subscription,c45582ad-79dc-42ef-8c43-496042f1e532
2021-08-05 19:25:41.218924+00:00,drift_magnitude,45ab47c1-6d1d-4ac1-8f00-f27c73bbdb46,0.06546341463414629,None,0.1,[],drift,ce617984-9b10-4a73-9fa6-8cfcaa027e49,1e9e9530-4679-48d3-a06a-a903b9279b4c,subscription,c45582ad-79dc-42ef-8c43-496042f1e532
2021-08-05 19:25:41.218924+00:00,predicted_accuracy,45ab47c1-6d1d-4ac1-8f00-f27c73bbdb46,0.7455365853658538,None,None,[],drift,ce617984-9b10-4a73-9fa6-8cfcaa027e49,1e9e9530-4679-48d3-a06a-a903b9279b4c,subscription,c45582ad-79dc-42ef-8c43-496042f1e532


### Enable Explainability and run explanation on sample record

In [78]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result




 Waiting for end of monitor instance creation 636ca910-7714-4fec-aa13-071d64d7c088 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




Getting a `transaction_id` to run explanation on

In [79]:
explainability_monitor_id = explainability_details.metadata.id
explainability_monitor_id

'636ca910-7714-4fec-aa13-071d64d7c088'

In [80]:
wos_client.data_sets.show_records(data_set_id=payload_data_set_id)

1.0,None,0.0,"[0.5598874688148499, 0.44011253118515015]",0.0,0.0,0.0,5308.0,97093668-aebf-4167-8988-d85ac19127a9-1,0.0,0.0,1.0,3.0,0,0.0,2021-08-05T19:24:01.277Z,0.0,1.0,0.0,0.5598874688148499,0.0,"[0.5598874688148499, 0.44011253118515015]",0.0,0.0,0.44011253118515015,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,38.0,31.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,44cbb38bb2c81f10236feaff987a5bb9,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0,0.0,0.0,0.0
0.0,None,0.0,"[0.9514117650687695, 0.048588234931230545]",0.0,0.0,0.0,620.0,97093668-aebf-4167-8988-d85ac19127a9-10,0.0,0.0,0.0,2.0,0,0.0,2021-08-05T19:24:01.277Z,0.0,1.0,0.0,0.9514117650687695,0.0,"[0.9514117650687695, 0.048588234931230545]",0.0,0.0,0.048588234931230545,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,12.0,30.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,44cbb38bb2c81f10236feaff987a5bb9,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0,0.0,0.0,0.0
0.0,None,0.0,"[0.8563538193702698, 0.14364618062973022]",0.0,0.0,0.0,1387.0,97093668-aebf-4167-8988-d85ac19127a9-11,0.0,0.0,1.0,2.0,0,0.0,2021-08-05T19:24:01.277Z,1.0,1.0,0.0,0.8563538193702698,0.0,"[0.8563538193702698, 0.14364618062973022]",0.0,0.0,0.14364618062973022,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,16.0,28.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,44cbb38bb2c81f10236feaff987a5bb9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0.0,0.0,0.0
1.0,None,0.0,"[0.5535019040107727, 0.4464980959892273]",0.0,0.0,0.0,6225.0,97093668-aebf-4167-8988-d85ac19127a9-12,0.0,0.0,1.0,3.0,0,0.0,2021-08-05T19:24:01.277Z,0.0,1.0,0.0,0.5535019040107727,0.0,"[0.5535019040107727, 0.4464980959892273]",0.0,0.0,0.4464980959892273,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,25.0,40.0,1.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,2.0,44cbb38bb2c81f10236feaff987a5bb9,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0,0.0,0.0,0.0
1.0,None,0.0,"[0.3814590573310852, 0.6185409426689148]",1.0,0.0,0.0,4424.0,97093668-aebf-4167-8988-d85ac19127a9-13,0.0,0.0,1.0,3.0,1,0.0,2021-08-05T19:24:01.277Z,0.0,1.0,0.0,0.6185409426689148,0.0,"[0.3814590573310852, 0.6185409426689148]",0.0,0.0,0.6185409426689148,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,32.0,32.0,0.0,1.0,4.0,1.0,0.0,0.0,0.0,0.0,2.0,44cbb38bb2c81f10236feaff987a5bb9,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1,0.0,0.0,0.0
0.0,None,0.0,"[0.9773842208087444, 0.02261577919125557]",0.0,1.0,0.0,2244.0,97093668-aebf-4167-8988-d85ac19127a9-14,0.0,0.0,1.0,2.0,0,0.0,2021-08-05T19:24:01.277Z,1.0,1.0,0.0,0.9773842208087444,0.0,"[0.9773842208087444, 0.02261577919125557]",0.0,0.0,0.02261577919125557,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,14.0,21.0,0.0,1.0,3.0,0.0,0.0,0.0,1.0,0.0,1.0,44cbb38bb2c81f10236feaff987a5bb9,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0.0,0.0,0.0
1.0,None,0.0,"[0.3769074082374573, 0.6230925917625427]",1.0,0.0,0.0,2992.0,97093668-aebf-4167-8988-d85ac19127a9-15,0.0,0.0,1.0,4.0,1,0.0,2021-08-05T19:24:01.277Z,0.0,1.0,0.0,0.6230925917625427,0.0,"[0.3769074082374573, 0.6230925917625427]",0.0,0.0,0.6230925917625427,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,53.0,0.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,2.0,44cbb38bb2c81f10236feaff987a5bb9,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,1.0
1.0,None,0.0,"[0.6244255900382996, 0.37557440996170044]",1.0,0.0,0.0,2034.0,97093668-aebf-4167-8988-d85ac19127a9-16,0.0,0.0,1.0,4.0,0,0.0,2021-08-05T19:24:01.277Z,0.0,1.0,0.0,0.6244255900382996,0.0,"[0.6244255900382996, 0.37557440996170044]",0.0,0.0,0.37557440996170044,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,47.0,0.0,1.0,3.0,0.0,0.0,0.0,1.0,1.0,2.0,44cbb38bb2c81f10236feaff987a5bb9,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0,0.0,0.0,1.0
0.0,None,0.0,"[0.9635847955942154, 0.03641520440578461]",0.0,1.0,0.0,2804.0,97093668-aebf-4167-8988-d85ac19127a9-17,0.0,0.0,1.0,3.0,0,0.0,2021-08-05T19:24:01.277Z,0.0,1.0,0.0,0

In [81]:
payload_data = wos_client.data_sets.get_list_of_records(limit=5,data_set_id=payload_data_set_id,output_type='pandas').result
scoring_ids=payload_data['scoring_id'].tolist()
print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types).result
print(result)
explanation_task_id=result.to_dict()['metadata']['explanation_task_ids'][0]
explanation_task_id

Running explanations on scoring IDs: ['97093668-aebf-4167-8988-d85ac19127a9-1', '97093668-aebf-4167-8988-d85ac19127a9-10', '97093668-aebf-4167-8988-d85ac19127a9-11', '97093668-aebf-4167-8988-d85ac19127a9-12', '97093668-aebf-4167-8988-d85ac19127a9-13']
{
  "metadata": {
    "explanation_task_ids": [
      "f1f0380e-fa15-4822-a493-44df67a62614",
      "ce9bbed9-6c47-4c09-9582-00aa326df1a0",
      "3d241fad-e7e3-4dbb-970a-9d3b619b8051",
      "9f376f22-41d9-4b75-9675-1717a3b62c53",
      "fa819977-cd41-4854-af99-efd0838df930"
    ],
    "created_by": "IBMid-50M71Q0T71",
    "created_at": "2021-08-05T19:26:57.559424Z"
  }
}


'f1f0380e-fa15-4822-a493-44df67a62614'

In [82]:
wos_client.monitor_instances.get_explanation_tasks(explanation_task_id=explanation_task_id).result.to_dict()

{'metadata': {'explanation_task_id': 'f1f0380e-fa15-4822-a493-44df67a62614',
  'created_by': 'IBMid-50M71Q0T71',
  'created_at': '2021-08-05T19:26:57.559424Z',
  'updated_at': '2021-08-05T19:27:01.715309Z'},
 'entity': {'status': {'state': 'in_progress'},
  'asset': {'id': '9345b2a3d122475ee4ea44be26d16df4',
   'name': 'Credit-risk-linear-learner-2021-08-05-17-39',
   'input_data_type': 'structured',
   'problem_type': 'binary',
   'deployment': {'id': '44cbb38bb2c81f10236feaff987a5bb9',
    'name': 'Credit-risk-endpoint-scoring-2021-08-05-17-39'}},
  'scoring_id': '97093668-aebf-4167-8988-d85ac19127a9-1'}}

## Congratulations

You have finished the tutorial for IBM Watson OpenScale and AWS Machine Learning Studio. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the German Credit AWS model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.

---